In [72]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, MaxPool2D,
    Dropout, Flatten, Dense, Add
)

import matplotlib.pyplot as plt
%matplotlib inline

from deepface import dataloader

## The FER Dataset (Train / Dev / Test sets)

In [73]:
train, dev, test = dataloader.train_dev_test_sets()

In [74]:
print(f'Train set size: {len(train[0])}')
print(f'Dev set size: {len(dev[0])}')
print(f'Test set size: {len(test[0])}')
print()

print(f'Image shape: {dataloader.IMG_SHAPE}')
print(f'Number of classes: {len(dataloader.LABELS_MAP.values())}')
print(f'Classes: {list(dataloader.LABELS_MAP.values())}')
print()

Train set size: 28709
Dev set size: 3589
Test set size: 3589

Image shape: (48, 48, 1)
Number of classes: 7
Classes: ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']



### Identity Residual Block

In [112]:
def identity_resblock(X, filters=[], f=3):
    F1, F2 = filters
    
    # Save the input
    X_input = X
    
    # Conv2d - 1
    X = Conv2D(F1, kernel_size=[f, f], strides=[1, 1], padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    # Conv2d - 2
    X = Conv2D(F2, kernel_size=[f, f], strides=[1, 1], padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    
    # Merge
    X = Add()([X, X_input])
    X = Activation('relu')(X)
    
    return X

### Convolution Residual Block

In [113]:
def conv_resblock(X, filters=[], f=3, s=2):
    F1, F2 = filters
    
    # Save the input
    X_input = X
    
    # Conv2d - 1
    X = Conv2D(F1, kernel_size=[1, 1], strides=[1, 1], padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    # Conv2d - 2
    X = Conv2D(F2, kernel_size=[f, f], strides=[s, s], padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    
    # Res Connection
    X_input = Conv2D(F2, kernel_size=[f, f], strides=[s, s], padding='valid')(X_input)
    X = BatchNormalization(axis=3)(X)
    
    # Merge
    X = Add()([X, X_input])
    X = Activation('relu')(X)
    
    return X

In [120]:
def init_model():
    # Input
    X_input = Input(shape=dataloader.IMG_SHAPE)
    
    X = conv_resblock(X_input, filters=[8, 8], f=3, s=2)
    X = identity_resblock(X_input, filters=[8, 16], f=3)
    X = identity_resblock(X_input, filters=[16, 16], f=3)
    
    X = conv_resblock(X, filters=[32, 32], f=5, s=2)
    X = identity_resblock(X_input, filters=[32, 64], f=7)
    X = identity_resblock(X_input, filters=[64, 64], f=7)
    
    X = Flatten()(X)
    
    X = Dense(units=100, activation='relu')(X)
    X = Dense(units=len(dataloader.LABELS_MAP.values()), activation='softmax')(X)
    
    return Model(inputs=X_input, outputs=X)

In [121]:
model = init_model()

In [122]:
optmizer = tf.train.AdamOptimizer(0.0001)
model.compile(optmizer, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [124]:
model.fit(x=train[0], y=train[1], batch_size=128, epochs=5,
          callbacks=[tf.keras.callbacks.ReduceLROnPlateau()])